# Script to compute ABAP card from eBird data

The aims of this code is to produce a dataset of ABAP full protocol card equivalent from the eBird EBD dataset.

## Overview rational of the approach

A card is uniquely defined by three elements: pentad*code, user_id, and date of the first day of the 5 days. We therefore use the `card_id = {pentad}*{observer}\_{date}`. From these three variables, it is possible to find all checklists that belong to it.

The aim here is to build a list of **valid** cards for which will then be used to find the `checklist_id` that belongs to each valid card and then finally compute the card info from the list of checklists.

Pentad: we need to assign for each checklist its pentad and check that the distance traveled is within the boundary of the pentad.
User_id is quite straightforward to build.
Date: much more challenging. See below for details.

General steps:

1. Construct the list of valid cards
   1. Group raw EBD data to checklist level information (merge shared checklist)
   2. Filter checklists which could make a valid card
      1. Keep only complete checklists
      2. Keep only checklists with `Historical`, `Stationary`, `Traveling`, `Incidental` protocol
      3. Keep only checklists within the pentad, that is,
         1. Exclude checklists with `historical` protocol that don't have a distance.
         2. Exclude checklists with distance greater than the distance from center of checklist to closest pentad limit (accept some overlap with a correction factor).
      4. Keep only checklists with duration greated than 0.
   3. Group checklists by date (named `checkday` later on)
   4. Group checklists into pentad_observer group so that we only have to loop through the date to find valid card
   5. Preliminary filter to eliminate all pentad_observer for which the sum over the entire period does not lead to 2h
   6. For each remaining pentad_observer, apply the function `checkday_pentad_observer()`, which,
      1. Compute the temporal distance between all checkday and check if they are within 5 days.
      2. Loop through all checkday,
         1. Compute the total duration of all checkdays within temporal distance
            1. If valid, create the card_id and apply it to all checkday. Iterate to the first next checkday that was not within temporal distance
            2. If invalid, iterate to the next checkday
2. Create the card data
   1. For each valid card, aggregate all checklists which are (1) within pentad, (2) same observer and (3) day within the 5 day period. This include more checklists than used to construct the list of valid cards
3. Add species level information to cards
   1. Add sequence information based on first occurance on checklist.
4. Export in JSON


## General set-up

### Load library


In [32]:
import pandas as pd
import numpy as np
import json
import os
import datetime
import requests
import tarfile
import sys
import csv

sys.path.append("../functions")

from latlng2pentad import latlng2pentad
from pentad2latlng import pentad2latlng

### Download the latest EBD AFRICA data


This code doesn't work as you need to sign in into to get access to the file.


In [33]:
# Calculate previous month and year
today = datetime.date.today()
last_month = today.replace(day=1) - datetime.timedelta(days=1)
year = last_month.strftime("%Y")
month = last_month.strftime("%b")

# Construct URL and filename
url = f"https://ebird.org/data/download?p=prepackaged/ebd_AFR_rel{month}-{year}.tar"
filename = os.path.basename(url)
filepath = os.path.join("../data/eBird/", filename)

# with open(filepath, "wb") as f:
#     f.write(requests.get(url).content)

In [34]:
# with tarfile.open(filepath, "r") as tar:
#     tar.extractall(f"../data/eBird/ebd_AFR_rel{month}-{year}/")

### Read the EBD file


In [35]:
year = "2024"
month = "Jul"

# file = "../data/eBird/ebd_AFR_rel{month}-{year}/ebd_AFR_rel{month}-{year}.txt.gz"
file = f"../data/eBird/ebd_AFR_rel{month}-{year}/ebd_AFR_rel{month}-{year}.txt.gz"
# file = "../data/eBird/ebd_AFR_relJul-2024/ebd_AFR_relJul-2024.txt.gz"


# For country specific EBD file, you can use:
# cntr = "KE"
# file = "../data/eBird/chk_{cntr}_relAug-2022/ebd_{cntr}_relAug-2022.txt"

In [36]:
ebd0 = pd.read_csv(
    file,
    delimiter="\t",
    usecols=[
        "SAMPLING EVENT IDENTIFIER",
        "SCIENTIFIC NAME",
        "CATEGORY",
        "LATITUDE",
        "LONGITUDE",
        "OBSERVATION DATE",
        "TIME OBSERVATIONS STARTED",
        "PROTOCOL TYPE",
        "DURATION MINUTES",
        "EFFORT DISTANCE KM",
        "ALL SPECIES REPORTED",
        "OBSERVER ID",
    ],
    parse_dates=["OBSERVATION DATE"],
)

KeyboardInterrupt: 

In [ ]:
ebd = ebd0

In [ ]:
# Create OBSERVATIONDATETIME by combining date and time
tmp = ebd["TIME OBSERVATIONS STARTED"].fillna("00:00:00")
ebd["OBSERVATION DATETIME"] = pd.to_datetime(
    ebd["OBSERVATION DATE"].dt.strftime("%Y-%m-%d") + " " + tmp,
    format="%Y-%m-%d %H:%M:%S",
)

# Sort by date: Important to have for filtering duplicate card-adu and needed for sequence
ebd.sort_values(by="OBSERVATION DATETIME", inplace=True)

# Keep only species category
# ebd0[["COMMONNAME", "SCIENTIFIC NAME", "CATEGORY"]].drop_duplicates().to_csv("species_list_ebird.csv", index=False)

# Keep some spuh which can be matched to an ADU
# spuh_keep = pd.read_csv("data/spuh_keep.csv", dtype=str)
# ebd0 = ebd0[(~ebd0["CATEGORY"].isin(["spuh", "slash"])) | ebd0["SCIENTIFIC NAME"].isin(spuh_keep["Clements--scientific_name"])]

In [ ]:
# Add Avibase taxon concept id and species code
species_ebird = pd.read_excel("../data/species_list/ebird_taxonomy_v2023.xlsx").rename(
    columns={
        "SCI_NAME": "SCIENTIFIC NAME",
        "TAXON_CONCEPT_ID": "TAXON CONCEPT",
        "SPECIES_CODE": "SPECIES CODE",
    }
)
ebd = pd.merge(
    ebd,  # .loc[:,['OBSERVER ID', 'PENTAD', "SAMPLING EVENT IDENTIFIER", "OBSERVATION DATE"]],
    species_ebird[["SCIENTIFIC NAME", "TAXON CONCEPT", "SPECIES CODE"]],
    how="left",
)

In [ ]:
# Read matched_species data. See species_match.ipynb
matched_species = pd.read_csv("../data/species_list/matched_species.csv")

ebd = pd.merge(
    ebd,  # .loc[:,['OBSERVER ID', 'PENTAD', "SAMPLING EVENT IDENTIFIER", "OBSERVATION DATE"]],
    matched_species[["TAXON CONCEPT", "ADU"]].drop_duplicates(subset="TAXON CONCEPT"),
    how="left",
)

In [ ]:
unmatched = (
    ebd[ebd["ADU"].isna()][["SCIENTIFIC NAME"]].value_counts().sort_values(ascending=0)
)
print(
    f"We still have {len(unmatched)} unmatched taxons, corresponding to {round(sum(unmatched)/len(ebd)*100)}% of our data"
)
unmatched

We still have 973 unmatched taxons, corresponding to 4% of our data


SCIENTIFIC NAME        
Icthyophaga vocifer        90635
Spermestes cucullata       82830
Zapornia flavirostra       51197
Lanius melanoleucus        34528
Ardea brachyrhyncha        31084
                           ...  
Egretta novaehollandiae        1
Aegithalos caudatus            1
Zenaida auriculata             1
Ducula bicolor                 1
Cacatua sulphurea              1
Name: count, Length: 973, dtype: int64

In [ ]:
print(f"# data: {len(ebd)}")
print(f"# Checklists: {len(ebd['SAMPLING EVENT IDENTIFIER'].unique())}")
print(f"# Species: {len(ebd['SCIENTIFIC NAME'].unique())}")

# data: 21190167
# Checklists: 1262874
# Species: 3127


## Build checklist dataframe


In [ ]:
chk = ebd[
    [
        "SAMPLING EVENT IDENTIFIER",
        "LATITUDE",
        "LONGITUDE",
        "OBSERVATION DATE",
        "OBSERVATION DATETIME",
        "PROTOCOL TYPE",
        "DURATION MINUTES",
        "EFFORT DISTANCE KM",
        "ALL SPECIES REPORTED",
        "OBSERVER ID",
    ]
].drop_duplicates()

# Sort by date
chk.sort_values(by="OBSERVATION DATETIME", inplace=True)

In [ ]:
# For some shared checklist some variable are different for the same sampling event.
# chk[chk["SAMPLING EVENT IDENTIFIER"].duplicated(keep=False)].sort_values(by="SAMPLING EVENT IDENTIFIER")
# ebd0[ebd0["SAMPLING EVENT IDENTIFIER"] == "S97700871"]
chk = chk.drop_duplicates("SAMPLING EVENT IDENTIFIER").reset_index(drop=True)
len(chk)

1262874

In [ ]:
# Filter protocol
chk["KEEP PROTOCOL"] = chk["PROTOCOL TYPE"].isin(
    ["Historical", "Incidental", "Stationary", "Traveling"]
)

In [ ]:
# Pentad
# Assign the pentad to all checklists based on their location
chk["PENTAD"] = latlng2pentad(chk["LATITUDE"], chk["LONGITUDE"])

# Retrieve the lat, lon center of the assigned pentad
lat, lon = pentad2latlng(chk["PENTAD"])

# Convert effort distance of the checklisst into degree lat-lon
effort_distance_lat = 180 / np.pi / 6371 * chk["EFFORT DISTANCE KM"]
effort_distance_lon = (
    180 / np.pi / 6371 / np.cos(np.radians(chk["LATITUDE"])) * chk["EFFORT DISTANCE KM"]
)

# Compute the distance from the center of the pentad (lat,lon) to the max distance possible if the observer traveled in the worst possible direction (i.e., to the closest eadge of the pentad)
# We relax a little bit the assumption of moving on the straight line to the edge of the pentad by applying a correction factor
corr_straight_line = 0.8
dist_lat = np.abs(lat - chk["LATITUDE"]) + effort_distance_lat * corr_straight_line
dist_lon = np.abs(lon - chk["LONGITUDE"]) + effort_distance_lon * corr_straight_line

# The maximum distance allowed for the checklist to be considered valid is half of a pentad resolution (5/60°)
# We accept that the checklist might have traveled a bit more that this distance
corr_overlap = 1.2  # allow for a 20% overlap
max_dist = (5 / 60 / 2) * corr_overlap

chk["KEEP PENTAD"] = (dist_lat < max_dist) & (dist_lon < max_dist)

# Filter historical checklists which have no distance
chk.loc[
    (chk["PROTOCOL TYPE"] == "Historical") & chk["EFFORT DISTANCE KM"].isna(),
    "KEEP PENTAD",
] = False

# Filter historical checklists which have no distance
chk.loc[
    chk["EFFORT DISTANCE KM"].isna(),
    "KEEP PENTAD",
] = True

chk.loc[
    (chk["PROTOCOL TYPE"] == "Historical") & chk["EFFORT DISTANCE KM"].isna(),
    "KEEP PENTAD",
] = False

## Find all possible valid card

Cards are considered to be full protocol if the sum of durations of the underlying checklists exceed 2 hours over the next rolling 5 days.
In this section, we first indentify which checklists can create a valid full card.


In [ ]:
# Find the index of all checklists which contribute to the 2hr rule. Note that we will still use "non-valid" checklists later as their species still contribute to the card.
valid_id = (
    chk["KEEP PENTAD"]
    & chk["KEEP PROTOCOL"]
    & (chk["DURATION MINUTES"] > 0)
    & chk["ALL SPECIES REPORTED"]
)

# Filter for valid checklist and create in a smaller table
check = chk.loc[
    valid_id, ["PENTAD", "OBSERVER ID", "OBSERVATION DATE", "DURATION MINUTES"]
]

# Combine checklists made by the same observer, pentad, and day. This is an intermediate step which enables us to grid the 5 days windows more easily
checkday = (
    check.groupby(["PENTAD", "OBSERVER ID", "OBSERVATION DATE"])
    .agg({"DURATION MINUTES": "sum"})
    .reset_index()
)

# Sort the checklist by date
checkday.sort_values(by=["OBSERVATION DATE"], inplace=True)

# Create additional columns
checkday["pentad_observer"] = checkday["PENTAD"] + "_" + checkday["OBSERVER ID"]
checkday["pentad_observer_date"] = (
    checkday["PENTAD"]
    + "_"
    + checkday["OBSERVER ID"].str[3:]
    + "_"
    + checkday["OBSERVATION DATE"].dt.strftime("%Y%m%d")
)

In [ ]:
# Do a first filter to eliminate all pentad_observer witout sufficient total duration time. (Aim is to just reduce the computation later)
pentad_observer_duration = checkday.groupby(["pentad_observer"])[
    "DURATION MINUTES"
].sum()
pentad_observer_duration_index = pentad_observer_duration[
    pentad_observer_duration >= 2 * 60
].index
checkday_long = checkday[
    checkday["pentad_observer"].isin(pentad_observer_duration_index)
]

# Second filter for reducing the test case
# pentad_observer_unique = checkday_long["pentad_observer"].unique()
# pentad_observer_unique = pentad_observer_unique[0:1000]
# checkday_long = checkday_long[checkday_long["pentad_observer"].isin(pentad_observer_unique)]

In [ ]:
# To find all pentad with sufficient duration effort (i.e, a sum of 2h over 5 days period), we apply this function for each pentad_observer.
def checkday_pentad_observer(df):
    df["CARD"] = ""
    # Build a matrix of distance between all checklists to check if they are close to each other
    di = np.abs(
        df["OBSERVATION DATE"].values[:, None] - df["OBSERVATION DATE"].values
    ) < pd.Timedelta(days=5)
    # create duration array to make computation slightly faster
    duration = df["DURATION MINUTES"].to_numpy()
    # Initie the card array with empty string
    # card = np.array(['' for x in range(len(df))], dtype='object')
    u = 1
    # Loop trough the list of checklists
    while u <= len(df):
        # Find all neighbord
        nb_neighbor = np.sum(di[u - 1, (u - 1) :])
        neigh = u + np.arange(0, nb_neighbor) - 1
        dur = duration[neigh].sum()
        # Check that total duration is more than 2hours, if so add card code (pentad_observer_date) to card array
        if dur >= (2 * 60):
            df.iloc[neigh, df.columns.get_loc("CARD")] = df.iloc[
                u - 1, df.columns.get_loc("pentad_observer_date")
            ]
        u += nb_neighbor
    return df

In [ ]:
# Apply the function defined above for each pentad-observer at the same time (makes operation much faster)
checkday_long_card = (
    checkday_long.groupby("pentad_observer")
    .apply(checkday_pentad_observer, include_groups=False)
    .reset_index()
)

In [ ]:
# Create the DataFrame of all valid card
card_valid = checkday_long_card[
    checkday_long_card["CARD"] == checkday_long_card["pentad_observer_date"]
][["PENTAD", "OBSERVER ID", "OBSERVATION DATE", "CARD"]]

# Sort by card
card_valid.sort_values(by="CARD", inplace=True)

In [ ]:
card_valid

,PENTAD,OBSERVER ID,OBSERVATION DATE,CARD
4,0000_0920,obsr431479,2022-08-10,0000_0920_r431479_20220810
5,0000_0920,obsr431479,2024-02-18,0000_0920_r431479_20240218
9,0000_1135,obsr344577,1997-07-22,0000_1135_r344577_19970722
10,0000_1140,obsr107968,1996-07-08,0000_1140_r107968_19960708
11,0000_1140,obsr167174,1996-07-08,0000_1140_r167174_19960708
...,...,...,...,...
164700,3720c0950,obsr1896095,2020-03-02,3720c0950_r1896095_20200302
164701,3720c0950,obsr433404,2024-05-29,3720c0950_r433404_20240529
164704,3720c0950,obsr490904,2020-03-02,3720c0950_r490904_20200302
164705,3720c0950,obsr536040,2020-03-02,3720c0950_r536040_20200302


## Create Card dataframe by aggregating all checklists

We take back `chk` where all checklists (i.e., including the incidentals, stationary, etc...) and find if they contribute to an existing full card.


In [ ]:
# Filter for checklist to keep: within pentad and and pentad and observer present in the valid card list
chk_keep = chk[
    (chk["KEEP PENTAD"])
    & (
        (chk["PENTAD"] + chk["OBSERVER ID"]).isin(
            (card_valid["PENTAD"] + card_valid["OBSERVER ID"])
        )
    )
]

In [ ]:
# Combine all possible checklits with the valid card based on observer and pentad.
# This will create duplicate checklist with all cards submitted by the same observer, same pentad, but any date
chk_card = pd.merge(
    chk_keep,  # .loc[:,['OBSERVER ID', 'PENTAD', "SAMPLING EVENT IDENTIFIER", "OBSERVATION DATE"]],
    card_valid,
    on=["OBSERVER ID", "PENTAD"],
    suffixes=("_chk", "_card"),
    how="left",
)

# Filter the checklist for checklist beeing within the 5 days of the card so that there will be a single checklist-card now
duration = (
    chk_card["OBSERVATION DATE_chk"] - chk_card["OBSERVATION DATE_card"]
).dt.days
chk_card = chk_card[(duration >= 0) & (duration < 5)]

In [ ]:
# Cretate the card list with all checklists that belong to it. Compute aggregated value of all checklists
card_chk = (
    chk_card.groupby("CARD")
    .agg(
        {
            "SAMPLING EVENT IDENTIFIER": list,
            "OBSERVATION DATETIME": ["min", "max"],
            "DURATION MINUTES": "sum",
            "EFFORT DISTANCE KM": "sum",
        }
    )
    .reset_index()
)
card_chk.columns = ["_".join(col).strip("_") for col in card_chk.columns.values]

# merge with the information contained in card_valid
card_chk = pd.merge(card_chk, card_valid, on="CARD", how="inner")

## Retrieve species information per card


In [ ]:
# Filter the full dataset to get only the checklist used in the card data
ebd_f = ebd.loc[
    ebd["SAMPLING EVENT IDENTIFIER"].isin(chk_card["SAMPLING EVENT IDENTIFIER"]),
    [
        "SAMPLING EVENT IDENTIFIER",
        "SCIENTIFIC NAME",
        "ADU",
        "OBSERVATION DATETIME",
        "LATITUDE",
        "LONGITUDE",
        "EFFORT DISTANCE KM",
    ],
]

# Add card_id
ebd_f = pd.merge(
    ebd_f,
    chk_card.loc[:, ["SAMPLING EVENT IDENTIFIER", "CARD"]],
    on="SAMPLING EVENT IDENTIFIER",
    how="left",
)

# Keep a unique list of card-species (remove duplicate species in the same card, keeping the first one in time)
ebd_f.sort_values(
    by="OBSERVATION DATETIME", inplace=True
)  # SHould have been done already above, but necessary for keep="first"

ebd_f_u = ebd_f.drop_duplicates(subset=["CARD", "ADU"], keep="first").copy()
ebd_f_u.reset_index(drop=True, inplace=True)

In [ ]:
# Compute the sequence of records based on datetime entry
ebd_f_u["SEQ"] = (
    ebd_f_u.groupby("CARD")["OBSERVATION DATETIME"].rank(method="min").astype(int)
)
# Not sure why, but fillina NA by nothing
ebd_f_u["EFFORT DISTANCE KM"] = ebd_f_u["EFFORT DISTANCE KM"].fillna("")

# Convert datetime to standard format
ebd_f_u["OBSERVATION DATETIME"] = ebd_f_u["OBSERVATION DATETIME"].dt.strftime(
    "%Y-%m-%dT%H:%M:%SZ"
)

In [ ]:
# Extract the species list per card as a cell for vectorized computation
card_sp = (
    ebd_f_u.groupby("CARD")[
        [
            "ADU",
            "SEQ",
            "LATITUDE",
            "LONGITUDE",
            "OBSERVATION DATETIME",
            "EFFORT DISTANCE KM",
        ]
    ]
    .agg(list)
    .reset_index()
)
card_sp

,CARD,ADU,SEQ,LATITUDE,LONGITUDE,OBSERVATION DATETIME,EFFORT DISTANCE KM
0,0000_0920_r431479_20220810,"[3852.0, 385.0, 1152.0, 576.0, 629.0, 510.0, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10...","[-0.0392179, -0.0392179, -0.0392179, -0.039217...","[9.340437, 9.340437, 9.340437, 9.340437, 9.340...","[2022-08-10T11:55:00Z, 2022-08-10T11:55:00Z, 2...","[0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, ..."
1,0000_0920_r431479_20240218,"[3852.0, 50.0, 394.0, 784.0, 846.0, 1089.0, 35...","[1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[-0.0392179, -0.0392179, -0.0392179, -0.039217...","[9.340437, 9.340437, 9.340437, 9.340437, 9.340...","[2024-02-18T12:20:00Z, 2024-02-18T12:20:00Z, 2...","[, , , , 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1...."
2,0000_1135_r344577_19970722,"[3888.0, 306.0, 283.0, 246.0, 394.0]","[1, 1, 1, 1, 1]","[-0.0686645, -0.0686645, -0.0686645, -0.068664...","[11.6070557, 11.6070557, 11.6070557, 11.607055...","[1997-07-22T11:00:00Z, 1997-07-22T11:00:00Z, 1...","[1.0, 1.0, 1.0, 1.0, 1.0]"
3,0000_1140_r107968_19960708,"[2798.0, 61.0, 113.0, 3180.0, 399.0, 352.0, 36...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-0.0700264, -0.0700264, -0.0700264, -0.070026...","[11.7131367, 11.7131367, 11.7131367, 11.713136...","[1996-07-08T16:30:00Z, 1996-07-08T16:30:00Z, 1...","[1.609, 1.609, 1.609, 1.609, 1.609, 1.609, 1.6..."
4,0000_1140_r167174_19960708,"[57.0, 754.0, 352.0, 1628.0, 399.0, 11491.0, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-0.0700264, -0.0700264, -0.0700264, -0.070026...","[11.7131367, 11.7131367, 11.7131367, 11.713136...","[1996-07-08T16:30:00Z, 1996-07-08T16:30:00Z, 1...","[1.609, 1.609, 1.609, 1.609, 1.609, 1.609, 1.6..."
...,...,...,...,...,...,...,...
62391,3720c0950_r1896095_20200302,"[933.0, nan, 9638.0]","[1, 1, 1]","[37.2771433, 37.2771433, 37.2771433]","[9.8745954, 9.8745954, 9.8745954]","[2020-03-02T07:30:00Z, 2020-03-02T07:30:00Z, 2...","[, , ]"
62392,3720c0950_r433404_20240529,"[2162.0, 940.0, 2742.0, 2838.0, 507.0, 379.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 11, 13, 13]","[37.2788336, 37.2788336, 37.2788336, 37.278833...","[9.8773313, 9.8773313, 9.8773313, 9.8773313, 9...","[2024-05-29T16:11:00Z, 2024-05-29T16:11:00Z, 2...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
62393,3720c0950_r490904_20200302,"[9638.0, nan, 933.0]","[1, 1, 1]","[37.2771433, 37.2771433, 37.2771433]","[9.8745954, 9.8745954, 9.8745954]","[2020-03-02T07:30:00Z, 2020-03-02T07:30:00Z, 2...","[, , ]"
62394,3720c0950_r536040_20200302,"[933.0, 9638.0, nan]","[1, 1, 1]","[37.2771433, 37.2771433, 37.2771433]","[9.8745954, 9.8745954, 9.8745954]","[2020-03-02T07:30:00Z, 2020-03-02T07:30:00Z, 2...","[, , ]"


## Export


In [ ]:
# Merge the card information by checklist and species into an export card dataframe
card_exp = pd.merge(
    card_chk,
    card_sp,
    on="CARD",
)

# Set some default values
card_exp["Protocol"] = "F"
card_exp["ObserverEmail"] = "kenyabirdmap@naturekenya.org"
card_exp["ObserverNo"] = "22829"

card_exp["Hour1"] = ""
card_exp["Hour2"] = ""
card_exp["Hour3"] = ""
card_exp["Hour4"] = ""
card_exp["Hour5"] = ""
card_exp["Hour6"] = ""
card_exp["Hour7"] = ""
card_exp["Hour8"] = ""
card_exp["Hour9"] = ""
card_exp["Hour10"] = ""
card_exp["InclNight"] = "0"
card_exp["AllHabitats"] = "0"

card_exp["TotalHours"] = round(card_exp["DURATION MINUTES_sum"] / 60, 2)
card_exp["TotalDistance"] = round(card_exp["EFFORT DISTANCE KM_sum"], 2)
card_exp["TotalSpp"] = card_exp["ADU"].apply(lambda x: len(x))
card_exp["StartDate"] = card_exp["OBSERVATION DATETIME_min"].dt.date.apply(str)
card_exp["EndDate"] = card_exp["OBSERVATION DATETIME_max"].dt.date.apply(str)
card_exp["StartTime"] = card_exp["OBSERVATION DATETIME_min"].dt.strftime("%H:%M")


# Function that generate species record to be used for each species of each card
def create_records(
    ADU, SEQ, LATITUDE, LONGITUDE, OBSERVATION_DATETIME, EFFORT_DISTANCE_KM, CARD
):
    return [
        {
            "Sequence": SEQ,
            "Latitude": LATITUDE,
            "Longitude": LONGITUDE,
            "Altitude": "",
            "CardNo": CARD,
            "Spp": ADU,
            "Accuracy": EFFORT_DISTANCE_KM * 1000,
            "SightingTime": OBSERVATION_DATETIME,
        }
        for ADU, SEQ, LATITUDE, LONGITUDE, OBSERVATION_DATETIME, EFFORT_DISTANCE_KM in zip(
            ADU, SEQ, LATITUDE, LONGITUDE, OBSERVATION_DATETIME, EFFORT_DISTANCE_KM
        )
    ]


# Apply the function
card_exp["records"] = card_exp.apply(
    lambda row: create_records(
        row["ADU"],
        row["SEQ"],
        row["LATITUDE"],
        row["LONGITUDE"],
        row["OBSERVATION DATETIME"],
        row["EFFORT DISTANCE KM"],
        row["CARD"],
    ),
    axis=1,
)

# Rename to match ABAP server input
card_exp = card_exp.rename(
    columns={
        "CARD": "CardNo",
        "PENTAD": "Pentad",
        "SAMPLING EVENT IDENTIFIER": "Checklists",
        "OBSERVER ID": "ObserverNoEbird",
        "SAMPLING EVENT IDENTIFIER_list": "Checklists",
    }
)

card_exp = card_exp.reindex(
    columns=[
        "Protocol",
        "ObserverEmail",
        "CardNo",
        "StartDate",
        "EndDate",
        "StartTime",
        "Pentad",
        "ObserverNo",
        "TotalHours",
        "Hour1",
        "Hour2",
        "Hour3",
        "Hour4",
        "Hour5",
        "Hour6",
        "Hour7",
        "Hour8",
        "Hour9",
        "Hour10",
        "TotalSpp",
        "InclNight",
        "AllHabitats",
        "Checklists",
        "TotalDistance",
        "ObserverNoEbird",
        "records",
    ]
)

card_exp

,Protocol,ObserverEmail,CardNo,StartDate,EndDate,StartTime,Pentad,ObserverNo,TotalHours,Hour1,...,Hour8,Hour9,Hour10,TotalSpp,InclNight,AllHabitats,Checklists,TotalDistance,ObserverNoEbird,records
0,F,kenyabirdmap@naturekenya.org,0000_0920_r431479_20220810,2022-08-10,2022-08-10,11:55,0000_0920,22829,2.68,,...,,,,19,0,0,"[S116860554, S116861241]",2.60,obsr431479,"[{'Sequence': 1, 'Latitude': -0.0392179, 'Long..."
1,F,kenyabirdmap@naturekenya.org,0000_0920_r431479_20240218,2024-02-18,2024-02-20,12:20,0000_0920,22829,3.55,,...,,,,35,0,0,"[S162608678, S162609512, S162830782, S16283088...",3.50,obsr431479,"[{'Sequence': 1, 'Latitude': -0.0392179, 'Long..."
2,F,kenyabirdmap@naturekenya.org,0000_1135_r344577_19970722,1997-07-22,1997-07-22,11:00,0000_1135,22829,2.00,,...,,,,5,0,0,[S13388886],1.00,obsr344577,"[{'Sequence': 1, 'Latitude': -0.0686645, 'Long..."
3,F,kenyabirdmap@naturekenya.org,0000_1140_r107968_19960708,1996-07-08,1996-07-08,16:30,0000_1140,22829,3.00,,...,,,,15,0,0,[S7529043],1.61,obsr107968,"[{'Sequence': 1, 'Latitude': -0.0700264, 'Long..."
4,F,kenyabirdmap@naturekenya.org,0000_1140_r167174_19960708,1996-07-08,1996-07-08,16:30,0000_1140,22829,3.00,,...,,,,15,0,0,[S7529228],1.61,obsr167174,"[{'Sequence': 1, 'Latitude': -0.0700264, 'Long..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62391,F,kenyabirdmap@naturekenya.org,3720c0950_r1896095_20200302,2020-03-02,2020-03-02,07:30,3720c0950,22829,20.00,,...,,,,3,0,0,[S146633506],0.00,obsr1896095,"[{'Sequence': 1, 'Latitude': 37.2771433, 'Long..."
62392,F,kenyabirdmap@naturekenya.org,3720c0950_r433404_20240529,2024-05-29,2024-06-01,16:11,3720c0950,22829,5.93,,...,,,,14,0,0,"[S178139251, S178166960, S178246101, S178605942]",5.40,obsr433404,"[{'Sequence': 1, 'Latitude': 37.2788336, 'Long..."
62393,F,kenyabirdmap@naturekenya.org,3720c0950_r490904_20200302,2020-03-02,2020-03-02,07:30,3720c0950,22829,20.00,,...,,,,3,0,0,[S65389557],0.00,obsr490904,"[{'Sequence': 1, 'Latitude': 37.2771433, 'Long..."
62394,F,kenyabirdmap@naturekenya.org,3720c0950_r536040_20200302,2020-03-02,2020-03-02,07:30,3720c0950,22829,20.00,,...,,,,3,0,0,[S65387334],0.00,obsr536040,"[{'Sequence': 1, 'Latitude': 37.2771433, 'Long..."


In [ ]:
json_data = card_exp.to_json(orient="records", indent=2)
with open(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
    "w",
) as f:
    f.write(json_data)

Export in better format


In [ ]:
card_chk.to_csv(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_cards.csv",
    index=False,
)

In [ ]:
ebd_f_u[["CARD", "ADU", "SEQ"]].to_csv(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_records.csv",
    index=False,
)

KeyboardInterrupt: 